In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import json
import numpy as np
import pandas as pd
# Plot Images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
plt.style.use('ggplot')
import matplotlib
# Set GPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# Plotly 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
cd ..

/home/anirudh/github/aip-eval


In [3]:
from studio.evaluation.keras import metrics, utils, visualizer
from studio.evaluation.keras.evaluators import CNNEvaluator
from studio.evaluation.keras.evaluators import Evaluator
from studio.evaluation.keras.evaluators import SequentialCNNEvaluator
from studio.evaluation.keras.evaluators import VisualQAEvaluator
from visual_qa.visual_qa import VisualQA

Using TensorFlow backend.
/home/anirudh/github/aip-eval/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/anirudh/github/aip-eval/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/anirudh/github/aip-eval/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/anirudh/github/aip-eval/.venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a syno

In [4]:
report_dir = 'consensus_evaluation_asl_v1_133_condtions_mode_slelection_non_strict/'
model_path = '/data/models/133_way_april_2019/3_ensemble/xception/model_max_acc_1gpu.h5'
visual_dictionary = '/home/anirudh/github/qa-data/data/visual_classifier/133_way_min_150_images_april_2019_model_map_diagnosis_nodes.json'
by_definition_csv = '/home/anirudh/github/qa-data/data/by-definition/by_def_reviewed_133_conditions.csv'
qa_data_json = '/home/anirudh/github/qa-data/data/testset/testset_asl_5sep19.json'
valid_evidence = '/home/anirudh/github/qa-data/data/by-definition/asl_valid_evidence.json'
visual_data_dir = '/data/datasets/visual_qa/all_cases_dup_removed_july_24_cropped'

In [5]:
cnn_evaluator = CNNEvaluator(
        concept_dictionary_path=None,
        custom_objects=None,
        concepts=None,
        model_path=model_path,
        batch_size=32,
        verbose=0)

/home/anirudh/github/aip-eval/.venv/lib/python3.6/site-packages/keras/engine/saving.py:310: UserWarning:

No training configuration found in save file: the model was *not* compiled. Compile it manually.



In [6]:
visual_qa_evaluator = VisualQAEvaluator(
    report_dir=report_dir,
    qa_data_json = qa_data_json,
    visual_dictionary = visual_dictionary,
    by_definition_csv = by_definition_csv,
    valid_evidence = valid_evidence
)

In [7]:
with open('/home/anirudh/github/qa-data/data/visual_classifier/133_way_min_150_images_april_2019_model_map_diagnosis_nodes.json') as f:
    n_way = json.load(f)
id_class_name_dict = {}
for _class in n_way:
    id_class_name_dict[_class['diagnosis_id']] = _class['diagnosis_name']
filtered_diagnosis_ids = [_class['diagnosis_id'] for _class in n_way]

In [8]:
len(filtered_diagnosis_ids)

133

In [9]:
filtered_testset = [case for case in visual_qa_evaluator.filtered_qa_data if case['diagnosis_id'] in filtered_diagnosis_ids]

In [10]:
#qa_data_json = '/home/anirudh/github/qa-data/data/testset/survey_testset_derm_24july19.json'
len(filtered_testset)

2920

In [11]:
'''with open('/home/anirudh/github/qa-data/data/testset/custom_survey_testset_derm_24july19.json', 'w') as f:
    json.dump(filtered_testset, f, indent=4)
'''

"with open('/home/anirudh/github/qa-data/data/testset/custom_survey_testset_derm_24july19.json', 'w') as f:\n    json.dump(filtered_testset, f, indent=4)\n"

In [12]:
main_case = set()
for case in filtered_testset:
    main_case.add(case['case_id'].split('_')[0])

In [13]:
len(main_case)

1843

In [14]:
visual_diagnosis_ids = list(visual_qa_evaluator.visual_qa.visual_diagnosis_ids)
labels = []
image_paths = []
for sample in filtered_testset:
    image_paths.append(os.path.join(visual_data_dir, sample["image_id"]))
    labels.append(visual_diagnosis_ids.index(sample['diagnosis_id']))
    
probs = cnn_evaluator.predict(image_paths)
labels = np.array(labels)

Making predictions from model  0
90/91 [============================>.] - ETA: 1s
Processing remainder: 8


In [15]:
visual_result = cnn_evaluator.get_metrics(probs, labels, top_k=5, concept_labels=visual_qa_evaluator.visual_qa.visual_diagnosis_ids)

In [16]:
cnn_evaluator.plot_top_k_accuracy()

In [17]:
cnn_evaluator.show_results(mode='average')

,id,accuracy_top_1,accuracy_top_2,accuracy_top_3,accuracy_top_4,accuracy_top_5,weighted_precision,sensitivity,precision,weighted_f1_score,number_of_samples,number_of_classes
0,model_max_acc_1gpu.h5,0.358,0.481,0.555,0.606,0.652,0.414,0.29,0.323,0.352,2920,133


In [18]:
visual_qa_evaluator.filtered_qa_data = filtered_testset

In [22]:
combination_result = visual_qa_evaluator.evaluate(probs, differential=True, strict_mode=False)

In [23]:
utils.compare_visual_by_definition_results(visual_result['average']['accuracy'] ,combination_result['average']['accuracy'] )

,Mode,top_1,top_2,top_3,top_4,top_5
0,Visual,0.358219,0.481164,0.555137,0.605822,0.652397
1,Visual_QA,0.359932,0.486301,0.561301,0.616781,0.659589


In [29]:
report_dir = 'consensus_evaluation_asl_v1_133_condtions_mode_selection_strict/'

In [30]:
visual_qa_evaluator_strict = VisualQAEvaluator(
    report_dir=report_dir,
    qa_data_json = qa_data_json,
    visual_dictionary = visual_dictionary,
    by_definition_csv = by_definition_csv,
    valid_evidence = valid_evidence
)

In [31]:
visual_qa_evaluator_strict.filtered_qa_data = filtered_testset

In [32]:
combination_result_strict = visual_qa_evaluator_strict.evaluate(probs, differential=True, strict_mode=True)

In [33]:
utils.compare_visual_by_definition_results(visual_result['average']['accuracy'], combination_result_strict['average']['accuracy'] )

,Mode,top_1,top_2,top_3,top_4,top_5
0,Visual,0.358219,0.481164,0.555137,0.605822,0.652397
1,Visual_QA,0.363356,0.493151,0.567123,0.623630,0.664041


In [34]:
utils.compare_visual_by_definition_results(combination_result['average']['accuracy'], combination_result_strict['average']['accuracy'] )

,Mode,top_1,top_2,top_3,top_4,top_5
0,Visual,0.359932,0.486301,0.561301,0.616781,0.659589
1,Visual_QA,0.363356,0.493151,0.567123,0.623630,0.664041
